# 📘 Notebook 02 · Planificación de Insumos e Inventario a partir del Forecast de Ventas

Este notebook reúne **dos etapas fundamentales** del modelo propuesto en tu proyecto de tesis:

---

# 🔵 **Etapa 2: Explosión de receta (Plato → Insumos)**  
Convierte las predicciones diarias de platos (`forecast_plato`) en la demanda diaria de insumos (`demanda_insumo`), usando la tabla de recetas.

---

# 🔵 **Etapa 3: Simulación de inventario + Generación de Órdenes de Compra**  
A partir de:

- inventario inicial  
- compras en tránsito  
- demanda futura  
- política de inventario (s, S)  
- lead time, MOQ, múltiplos y capacidad  

El sistema calcula:

- **Inventario proyectado día a día**  
- **Órdenes de compra sugeridas**

---

# 🎯 **Salidas clave del notebook**

- `demanda_insumo_diaria_*.csv`
- `inventario_proyectado_*.csv`
- `planificacion_detalle_insumo_*.csv`

Estas se guardan en:

```
ASE/Evidencia/Modelo_Salidas/
```

---



# 1️⃣ Importación de librerías y configuración de rutas

Las rutas siguen la estructura oficial de tu proyecto:

```
data/raw/
data/processed/
ASE/Evidencia/Modelo_Salidas/
```

Si cambias algo, ajusta aquí.



In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from math import ceil

BASE_DIR = Path("../..").resolve()
DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROC = BASE_DIR / "data" / "processed"
FORECAST_DIR = BASE_DIR / "Evidencia" / "Modelo_Salidas"

print("BASE_DIR:", BASE_DIR)
print("DATA_RAW:", DATA_RAW)
print("FORECAST_DIR:", FORECAST_DIR)



BASE_DIR: C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion
DATA_RAW: C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\data\raw
FORECAST_DIR: C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\Evidencia\Modelo_Salidas


# 2️⃣ Selección del archivo de forecast a utilizar

Aquí eliges qué modelo será la **base de la planificación de compras**:

- `result_forecast_plato_xg.csv`
- `result_forecast_plato_rf.csv`
- `result_forecast_plato_lstm.csv`

Puedes alternar fácilmente para comparar modelos.



In [40]:
forecast_file = "result_forecast_plato_xg.csv"
# forecast_file = "result_forecast_plato_rf.csv"
# forecast_file = "result_forecast_plato_lstm.csv"

forecast_plato = pd.read_csv(
    FORECAST_DIR / forecast_file,
    parse_dates=["fecha"]
)

forecast_plato.columns = [c.lower() for c in forecast_plato.columns]

rename_map = {
    "y_hat_xgb": "demanda_predicha",
    "y_hat": "demanda_predicha",
    "prediccion": "demanda_predicha"
}

forecast_plato = forecast_plato.rename(columns=rename_map)

assert {"fecha", "id_plato", "demanda_predicha"}.issubset(forecast_plato.columns)
forecast_plato.head()



,fecha,id_plato,demanda_predicha
0,2026-01-01,1,19.877472
1,2026-01-01,2,18.862059
2,2026-01-01,3,14.478246
3,2026-01-01,4,17.427208
4,2026-01-01,5,12.735395


In [41]:
len(forecast_plato)

63

# 3️⃣ Carga de tablas origen del Data Generator

Estas tablas alimentan toda la planificación:

- `receta.csv` → define cuántos insumos requiere cada plato  
- `inventario_diario.csv` → stock histórico  
- `compras.csv` → pedidos históricos y en tránsito  
- `proveedor_x_insumo.csv` → lead time, MOQ, múltiplos  
- `almacen.csv` → capacidad máxima por insumo  



In [42]:
receta = pd.read_csv(DATA_RAW / "receta.csv")
inventario_diario = pd.read_csv(DATA_RAW / "inventario_diario.csv", parse_dates=["fecha"])
compras = pd.read_csv(DATA_RAW / "compras.csv", parse_dates=["fecha_registro", "fecha_llegada"])
proveedor_x_insumo = pd.read_csv(DATA_RAW / "proveedor_x_insumo.csv")

almacen_path = DATA_RAW / "almacen.csv"
almacen = pd.read_csv(almacen_path) if almacen_path.exists() else None

receta.head()



,id_receta,id_plato,id_insumo,cantidad,unidad_de_medida,observaciones
0,1,1,101,0.18,kg,NaN
1,2,1,102,0.15,kg,NaN
2,3,1,103,0.04,kg,NaN
3,4,1,116,0.01,kg,NaN
4,5,1,117,0.02,L,NaN


# 4️⃣ Explosión de recetas: Plato → Insumos

Este paso multiplica:

```
demanda_predicha_plato × cantidad_insumo
```

y consolida por:

- fecha
- id_insumo

### Resultado:
`demanda_insumo_diaria.csv`



In [43]:
assert {"id_plato", "id_insumo"}.issubset(receta.columns)

if "cantidad_insumo" not in receta.columns:
    cand = [c for c in receta.columns if "cant" in c.lower()]
    receta = receta.rename(columns={cand[0]: "cantidad_insumo"})

forecast_plato["id_plato"] = forecast_plato["id_plato"].astype(int)
receta["id_plato"] = receta["id_plato"].astype(int)
receta["id_insumo"] = receta["id_insumo"].astype(int)

PLANNING_START_DATE = forecast_plato["fecha"].min()
PLANNING_END_DATE = forecast_plato["fecha"].max()

df_req = forecast_plato.merge(receta, on="id_plato", how="left")
df_req["demanda_insumo"] = df_req["demanda_predicha"] * df_req["cantidad_insumo"]

req_insumo = df_req.groupby(["fecha","id_insumo"], as_index=False)["demanda_insumo"].sum()

req_insumo.head()



,fecha,id_insumo,demanda_insumo
0,2026-01-01,101,6.611510
1,2026-01-01,102,4.491051
2,2026-01-01,103,1.374229
3,2026-01-01,104,7.544823
4,2026-01-01,105,3.772412


In [44]:
len(req_insumo)

133

In [45]:
out_demanda_insumo = FORECAST_DIR / f"demanda_insumo_diaria_{forecast_file.replace('.csv','')}.csv"
req_insumo.to_csv(out_demanda_insumo, index=False)
out_demanda_insumo



WindowsPath('C:/Users/josep/OneDrive/Desktop/PUCP/2025-2/Tesis2/Resultados esperados/RESULTADO 3/Modelo/-1INF46-Plan_Compras_Produccion/Evidencia/Modelo_Salidas/demanda_insumo_diaria_result_forecast_plato_xg.csv')

# 5️⃣ Construcción del estado inicial de inventario

Se define:

- **stock inicial** → último inventario antes del inicio del horizonte
- **compras en tránsito** → órdenes con llegada futura

Esto es el “punto de partida” del almacén para empezar la simulación.



In [46]:
mask_inv = inventario_diario["fecha"] <= PLANNING_START_DATE
inv_ref_date = inventario_diario.loc[mask_inv, "fecha"].max()

inv_ref = (
    inventario_diario[inventario_diario["fecha"] == inv_ref_date]
    .groupby("id_insumo", as_index=False)["stock_utilizable"]
    .sum()
    .rename(columns={"stock_utilizable": "stock_on_hand"})
)

compras_futuras = compras[compras["fecha_llegada"] >= PLANNING_START_DATE]
inv_ref.head()



,id_insumo,stock_on_hand
0,102,3.521564
1,105,0.318892
2,107,6.220742
3,109,0.926487
4,110,4.090919


# 6️⃣ Parámetros del modelo de inventario (s, S)

Aquí calculamos:

- demanda promedio diaria  
- desviación estándar  
- demanda durante el lead time  
- buffer de seguridad  
- punto de reorden (s)  
- nivel objetivo (S)  
- restricciones de proveedor:
  - MOQ  
  - múltiplos  
  - lead time  



In [47]:
# ========================
# 1) Calcular lead time real por proveedor + insumo
# ========================

# Asegurar que las fechas son datetime
compras["fecha_registro"] = pd.to_datetime(compras["fecha_registro"], errors="coerce")
compras["fecha_llegada"] = pd.to_datetime(compras["fecha_llegada"], errors="coerce")

# Lead time real en días (float)
compras["lead_time_real"] = (compras["fecha_llegada"] - compras["fecha_registro"]).dt.days

# Filtramos casos válidos
compras_validas = compras.dropna(subset=["lead_time_real", "id_insumo", "id_proveedor"])

# ========================
# 2) Promedio del lead time histórico
# ========================

leadtime_hist = (
    compras_validas.groupby(["id_insumo", "id_proveedor"], as_index=False)["lead_time_real"]
    .mean()
    .rename(columns={"lead_time_real": "lead_time"})
)

leadtime_hist["lead_time"] = leadtime_hist["lead_time"].clip(lower=1).astype(int)

leadtime_hist.head()


,id_insumo,id_proveedor,lead_time
0,101,1.0,4
1,101,3.0,7
2,101,5.0,6
3,101,6.0,2
4,101,7.0,5


In [48]:
assert {"id_insumo", "id_proveedor"}.issubset(proveedor_x_insumo.columns)

# Si en el futuro agregas moq / múltiplos, esto los usa; si no, crea columnas dummy
if "moq" not in proveedor_x_insumo.columns:
    proveedor_x_insumo["moq"] = 0.0
if "multiplo_pedido" not in proveedor_x_insumo.columns:
    proveedor_x_insumo["multiplo_pedido"] = 1.0

# Traemos el lead_time histórico a la matriz proveedor_x_insumo
pref_prov = proveedor_x_insumo.merge(
    leadtime_hist,
    on=["id_insumo", "id_proveedor"],
    how="left"
)

# Si no hay histórico para alguna combinación, asumimos lead_time = 2 días
pref_prov["lead_time"] = pref_prov["lead_time"].fillna(2).astype(int)

# 👉 AQUÍ elegimos el "mejor" proveedor: el de menor lead_time por insumo
pref_prov = (
    pref_prov
    .sort_values(["id_insumo", "lead_time"])  # menor lead_time primero
    .groupby("id_insumo", as_index=False)
    .first()  # se queda con el proveedor de menor lead time
)

# Estandarizamos los nombres
pref_prov = pref_prov.rename(columns={
    "id_proveedor": "id_proveedor_pref",
    "moq": "moq_pref",
    "multiplo_pedido": "multiplo_pref"
})

# Nos quedamos solo con lo que usará el modelo de inventario
pref_prov = pref_prov[[
    "id_insumo",
    "id_proveedor_pref",
    "lead_time",
    "moq_pref",
    "multiplo_pref"
]]

pref_prov.head()

,id_insumo,id_proveedor_pref,lead_time,moq_pref,multiplo_pref
0,101,6,2,0.0,1.0
1,102,2,2,0.0,1.0
2,103,4,3,0.0,1.0
3,104,4,2,0.0,1.0
4,105,1,2,0.0,1.0


# 7️⃣ Capacidad del almacén (opcional)

Si existe `almacen.csv`, se aplica una capacidad máxima por insumo.

Esto evita recomendar compras por encima del espacio real disponible.



In [49]:
if almacen is not None and "capacidad_maxima" in almacen.columns:
    col = [c for c in almacen.columns if "insumo" in c.lower()]
    capacidad_insumo = almacen.rename(columns={col[0]: "id_insumo"})[["id_insumo","capacidad_maxima"]]
else:
    capacidad_insumo = None

capacidad_insumo



# 8️⃣ Estado del sistema (stock + órdenes en tránsito + parámetros)

Se arma un diccionario `state` con **toda la información necesaria** para simular el inventario:

- stock actual  
- órdenes por llegar  
- s, S  
- lead time  
- proveedor  
- capacidad del almacén  



In [50]:
# 1) Estadísticas de demanda diaria pronosticada por insumo
stats_insumo = (
    req_insumo.groupby("id_insumo")["demanda_insumo"]
    .agg(["mean", "std"])
    .reset_index()
    .rename(columns={"mean": "mu_dia", "std": "sigma_dia"})
)

# 2) Combinar stats_insumo con pref_prov (proveedor, lead_time, moq, múltiplo)
params_insumo = stats_insumo.merge(pref_prov, on="id_insumo", how="left")

# 3) Limpieza / defaults
params_insumo["sigma_dia"] = params_insumo["sigma_dia"].fillna(0.0)
params_insumo["lead_time"] = params_insumo["lead_time"].fillna(1).astype(int)
params_insumo["moq_pref"] = params_insumo["moq_pref"].fillna(0.0)
params_insumo["multiplo_pref"] = params_insumo["multiplo_pref"].replace(0, 1.0)

# 4) Cálculo de s y S
SERVICE_Z = 1.65
SAFETY_DAYS = 2

params_insumo["demanda_LT"] = params_insumo["mu_dia"] * (params_insumo["lead_time"] + SAFETY_DAYS)
params_insumo["buffer"] = SERVICE_Z * params_insumo["sigma_dia"] * np.sqrt(params_insumo["lead_time"])
params_insumo["s"] = params_insumo["demanda_LT"] + params_insumo["buffer"]
params_insumo["S"] = 2 * params_insumo["demanda_LT"] + params_insumo["buffer"]

params_insumo.head()


,id_insumo,mu_dia,sigma_dia,id_proveedor_pref,lead_time,moq_pref,multiplo_pref,demanda_LT,buffer,s,S
0,101,6.723268,0.445221,6,2,0.0,1.0,26.893072,1.038901,27.931973,54.825045
1,102,4.600752,0.224072,2,2,0.0,1.0,18.403007,0.522860,18.925867,37.328875
2,103,1.452909,0.103727,4,3,0.0,1.0,7.264547,0.296441,7.560988,14.825535
3,104,7.779665,0.898355,4,2,0.0,1.0,31.118661,2.096268,33.214929,64.333591
4,105,3.889833,0.449177,1,2,0.0,1.0,15.559331,1.048134,16.607465,32.166795


In [51]:
cap_map = dict(zip(capacidad_insumo["id_insumo"], capacidad_insumo["capacidad_maxima"])) if capacidad_insumo is not None else {}
inv_map = dict(zip(inv_ref["id_insumo"], inv_ref["stock_on_hand"]))
params_map = params_insumo.set_index("id_insumo").to_dict(orient="index")

state = {}

for insumo_id in req_insumo["id_insumo"].unique():
    stock0 = inv_map.get(insumo_id, 0.0)
    p = params_map.get(insumo_id, {})
    on_order = []

    if insumo_id in compras_futuras["id_insumo"].unique():
        sub = compras_futuras[compras_futuras["id_insumo"] == insumo_id]
        for _, r in sub.iterrows():
            on_order.append({
                # usamos nombre estándar interno: fecha_entrega
                "fecha_entrega": r["fecha_llegada"].date(),   # o r["fecha_entrega"] si así se llama
                "cantidad": float(r["cantidad_entrada"]),     # estandarizamos como "cantidad"
                "id_proveedor": r.get("id_proveedor"),
                "es_historica": True
            })

    state[insumo_id] = {
        "stock_on_hand": stock0,
        "on_order": on_order,
        "s": p.get("s", 0.0),
        "S": p.get("S", 0.0),
        "lead_time": p.get("lead_time", 1),
        "moq": p.get("moq_pref", 0.0),
        "multiplo": p.get("multiplo_pref", 1.0),
        "id_proveedor_pref": p.get("id_proveedor_pref"),
        "capacidad_max": cap_map.get(insumo_id)
    }

len(state)




19

# 9️⃣ Simulación diaria de inventario

Para cada día:

1. Se reciben compras si llegan hoy  
2. Se consume lo correspondiente al forecast  
3. Se calcula posición = stock + pedidos en tránsito  
4. Si posición < s → se genera orden que eleva a S  
5. Se respeta:
   - MOQ
   - múltiplos
   - capacidad máxima  
   - lead time  

Todo se registra en:

- `inv_log_rows` → inventario día a día  
- `plan_rows` → órdenes generadas  



In [52]:
# Mapa (id_insumo, fecha) -> demanda del día
req_dict = {
    (r["id_insumo"], r["fecha"].date()): r["demanda_insumo"]
    for _, r in req_insumo.iterrows()
}

# Rango de fechas a simular: desde la primera hasta la última fecha del forecast
date_range = pd.date_range(PLANNING_START_DATE, PLANNING_END_DATE, freq="D")

# Listas donde se irá guardando:
# - las órdenes generadas
# - el log del inventario proyectado
plan_rows = []
inv_log_rows = []


In [53]:
for current_date in date_range:
    d = current_date.date()

    for iid, st in state.items():

        # 1. Recibir órdenes
        new_orders = []
        for o in st["on_order"]:
            if o["fecha_entrega"] == d:
                st["stock_on_hand"] += o["cantidad"]
            else:
                new_orders.append(o)
        st["on_order"] = new_orders

        # 2. Consumo del día
        demand = req_dict.get((iid, d), 0.0)
        st["stock_on_hand"] = max(st["stock_on_hand"] - demand, 0)

        # 3. posición
        pos = st["stock_on_hand"] + sum(o["cantidad"] for o in st["on_order"])
        s = st["s"]; S = st["S"]
        cap = st["capacidad_max"]
        Q = 0

        # 4. ¿Se genera orden?
        if pos < s:
            Q = S - pos

            if st["moq"] > 0:
                Q = max(Q, st["moq"])

            if st["multiplo"] > 1:
                Q = ceil(Q / st["multiplo"]) * st["multiplo"]

            if cap is not None:
                already = st["stock_on_hand"] + sum(o["cantidad"] for o in st["on_order"])
                espacio = cap - already
                Q = max(0, min(Q, espacio))

            if Q > 0:
                fecha_entrega = (current_date + pd.Timedelta(days=st["lead_time"])).date()
                prov = st["id_proveedor_pref"]

                st["on_order"].append({
                    "fecha_entrega": fecha_entrega,
                    "cantidad": Q,
                    "id_proveedor": prov,
                    "es_historica": False
                })

                plan_rows.append({
                    "id_insumo": iid,
                    "id_proveedor": prov,
                    "fecha_orden": d,
                    "fecha_entrega": fecha_entrega,
                    "cantidad": Q,
                    "motivo": "reposición_sS",
                    "forecast_file": forecast_file
                })

        inv_log_rows.append({
            "fecha": d,
            "id_insumo": iid,
            "stock_on_hand": st["stock_on_hand"],
            "posicion_inventario": st["stock_on_hand"] + sum(o["cantidad"] for o in st["on_order"]),
            "demanda_dia": demand,
            "Q_ordenado": Q,
            "capacidad_max": cap
        })


In [55]:
planificacion_detalle_insumo.head()

,id_insumo,id_proveedor,fecha_orden,fecha_entrega,cantidad,motivo,forecast_file
0,101,6,2026-01-01,2026-01-03,54.825045,reposición_sS,result_forecast_plato_xg.csv
1,102,2,2026-01-01,2026-01-03,24.412254,reposición_sS,result_forecast_plato_xg.csv
2,103,4,2026-01-01,2026-01-04,14.825535,reposición_sS,result_forecast_plato_xg.csv
3,104,4,2026-01-01,2026-01-03,64.333591,reposición_sS,result_forecast_plato_xg.csv
4,106,8,2026-01-01,2026-01-05,18.440177,reposición_sS,result_forecast_plato_xg.csv


# 🔟 Exportación de resultados finales

Se generan:

### ✔ `inventario_proyectado_*.csv`
Evolución completa del inventario por día y por insumo.

### ✔ `planificacion_detalle_insumo_*.csv`
Todas las órdenes sugeridas automáticamente por el modelo.

Ambos quedan como evidencia formal en:

```
ASE/Evidencia/Modelo_Salidas/
```



In [54]:
planificacion_detalle_insumo = pd.DataFrame(plan_rows)
inventario_proyectado = pd.DataFrame(inv_log_rows)

out_plan = FORECAST_DIR / f"planificacion_detalle_insumo_{forecast_file.replace('.csv','')}.csv"
out_inv  = FORECAST_DIR / f"inventario_proyectado_{forecast_file.replace('.csv','')}.csv"

planificacion_detalle_insumo.to_csv(out_plan, index=False)
inventario_proyectado.to_csv(out_inv, index=False)

out_plan, out_inv



(WindowsPath('C:/Users/josep/OneDrive/Desktop/PUCP/2025-2/Tesis2/Resultados esperados/RESULTADO 3/Modelo/-1INF46-Plan_Compras_Produccion/Evidencia/Modelo_Salidas/planificacion_detalle_insumo_result_forecast_plato_xg.csv'),
 WindowsPath('C:/Users/josep/OneDrive/Desktop/PUCP/2025-2/Tesis2/Resultados esperados/RESULTADO 3/Modelo/-1INF46-Plan_Compras_Produccion/Evidencia/Modelo_Salidas/inventario_proyectado_result_forecast_plato_xg.csv'))